# Imports


In [ ]:
import matplotlib.pyplot as plt

from astropy.stats import sigma_clipped_stats
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
import joblib
import numpy as np

spits_cutout = joblib.load('./datasets/spits_cutout_x2828_y5085.joblib')
spits_coords = joblib.load('./datasets/coords_spits_cutout_x2828_y5085.joblib')
spits_cutout_reshaped = spits_cutout.reshape(1,-1)[0]

import numpy as np

import matplotlib.pyplot as plt

from astropy.visualization import SqrtStretch

from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder

In [ ]:
def cartesionGrid(full_data, cutout_data):
    from sklearn.utils import extmath

    y1, x1 = np.where(full_data == cutout_data[0, 0])
    y1 = y1[0]
    x1 = x1[0]
    first_corner = y1, x1

    diag_corner = np.add(first_corner, cutout_data.shape)
    coords_x = np.array(range(first_corner[1], diag_corner[1]))
    coords_y = np.array(range(first_corner[0], diag_corner[0]))

    return extmath.cartesian([coords_x, coords_y]) # gives every tuple combination of x and y values; i.e. all points

# def get_spits_data():
#     from astropy.io import fits
#     from computer_path import FullMaps

#     return fits.getdata(FullMaps.Spitzer())
def get_spits_data():
    return joblib.load('./datasets/spits_data.joblib')


def cartesionGrid_spits(cutout_data):
    return cartesionGrid(get_spits_data(), cutout_data)
                    

# Creating mask

In [ ]:
mean, median, std = sigma_clipped_stats(spits_cutout, sigma=3.0)  
print((mean, median, std))

In [ ]:
def mask_with_dao(input_data):
    mean, median, std = sigma_clipped_stats(input_data, sigma=3.0)  
    d = DAOStarFinder(fwhm=10.0, threshold=5.*std)
    s = d(input_data - median)
    for col in s.colnames:
        s[ col ].info.format = '%.8g'
    
    return s, d

sources, daofind = mask_with_dao(spits_cutout)


In [ ]:
print(f'Number of stars found = {len(sources)  } ')

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())

plt.imshow(spits_cutout, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)

# Background Masking
Let's get rid of the background noise

In [ ]:
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(spits_cutout, norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')

In [ ]:
from astropy.stats import SigmaClip
from photutils.segmentation import detect_threshold, detect_sources
from photutils.utils import circular_footprint

## creating the source mask

In [ ]:
SIGMA = 10.90


def mask_sources(input_data, S):
    sigma_clip = SigmaClip(sigma=S, maxiters=10)
    threshold = detect_threshold(input_data, nsigma=6.250, sigma_clip=sigma_clip)
    segment_img = detect_sources(input_data, threshold, npixels=5)
    footprint = circular_footprint(radius=1)
    mask = segment_img.make_source_mask(footprint=footprint)

    return mask

source_mask = mask_sources(spits_cutout, SIGMA)
mean, median, std = sigma_clipped_stats(spits_cutout, sigma=5.0, mask=source_mask)
print(f'{mean, median, std = }')

## Creating a Background2D

In [ ]:
from photutils.background import Background2D, MedianBackground
sigma_clip = SigmaClip(sigma=3.)
bkg_estimator = MedianBackground()
bkg = Background2D(spits_cutout, (50,50), filter_size=(3,3), sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)

In [ ]:
print(f'{bkg.background_median = }\n{bkg.background_rms_median = }')

In [ ]:
spits_cutout_masked = spits_cutout - bkg.background
spits_double_masked = spits_cutout_masked - source_mask


In [ ]:
%matplotlib widget
plt.subplot(131)
plt.imshow(spits_cutout_masked, norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')
plt.subplot(132)
plt.imshow(spits_double_masked, norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')
plt.subplot(133)
plt.tight_layout()
plt.imshow(spits_cutout)

Look's like background was successfully masked

In [ ]:
sources = daofind(spits_cutout_masked - median)
for col in sources.colnames:
    sources[ col ].info.format = '%.8g'
sources

# Actual individual cutouts

In [ ]:

def sort_points_to_cutouts(sources_list, input_data):
    from astropy.nddata import Cutout2D

    x = sources_list['xcentroid']
    y = sources_list['ycentroid']
    points = list(zip(x, y))

    cutouts = []
    for p_index, point in enumerate( points ):
        if p_index > 10:
            c = Cutout2D(input_data, point, (50, 50)).data
            cutouts.append(c) 

    return np.array(cutouts)
    


In [ ]:

masked_cutouts = sort_points_to_cutouts(sources, spits_cutout_masked)


In [ ]:
%matplotlib inline
target_cutout = masked_cutouts[0]
plt.imshow(target_cutout, norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')

## Setting up data

In [ ]:
from astropy.nddata import Cutout2D
import pandas as pd
x = sources['xcentroid']
y = sources['ycentroid']
points = list(zip(x, y))

cutouts = []
cutouts_1d = []
for p_index, point in enumerate( points ):
    if p_index > 10:
        c = Cutout2D(spits_cutout_masked, point, (50, 50)).data

        cutouts.append(c)
        cutouts_1d.append(c.flatten())


cutouts = np.array(cutouts)
cutouts_1d = np.array(cutouts_1d)


## Masking cutouts

Just want to mask the peaks of the stars, to simulate the supersaturated data of other Spitzer points

In [ ]:
masked_cutouts = []
masked_cutouts_1d = []
for c in cutouts:
    sigma_clip = SigmaClip(sigma=7., maxiters=100)
    threshold = detect_threshold(c, nsigma=3.990, sigma_clip=sigma_clip)
    segment_img = detect_sources(c, threshold, npixels=5)
    footprint = circular_footprint(radius=1)
    mask = segment_img.make_source_mask(footprint=footprint)
    mean, median, std = sigma_clipped_stats(c, sigma=5.0, mask=mask)
    # print(f'{mean, median, std = }')
    m = c - mask - median 
    masked_cutouts.append(m)
    masked_cutouts_1d.append(m.flatten())

masked_cutouts = np.array(masked_cutouts)
masked_cutouts_1d = np.array(masked_cutouts_1d)

%matplotlib inline
plt.subplot(121)
plt.imshow(cutouts[0], norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')
plt.subplot(122)
plt.imshow(masked_cutouts[5], norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')

In [ ]:

print(f'training data size {masked_cutouts.shape}')
print('65 (50, 50) masked_cutouts ')
print(f'training data size {masked_cutouts_1d.shape}')


Just checking the cutouts

In [ ]:
col_size = 11 # limit so we don't get spammed with hundreds of images
for plot_index, c in enumerate( masked_cutouts ):
    plot_index += 1
    if plot_index < col_size:
        plt.subplot(1, col_size, plot_index)
        plt.imshow(c, norm=norm, origin='lower', cmap='Greys_r', interpolation='nearest')

# KNN

In [ ]:
input_train, input_test, output_train, output_test = train_test_split(masked_cutouts_1d, cutouts_1d, test_size=0.2)

In [ ]:
input_test.shape, output_test.shape

In [ ]:
kde = KNeighborsRegressor()
kde.fit(input_train, output_train)

kde_pred = kde.predict(input_test)

In [ ]:
%matplotlib inline
pred_img = kde_pred[10].reshape(50,50)
plt.subplot(121)
plt.title('Predicted')
plt.imshow(pred_img)
plt.subplot(122)
plt.title('Training Data')
plt.imshow(input_train[0].reshape(50,50))
